In [ ]:
from lab07 import *
from lab07_extra import *

# Linked List Practice

## Q9: Remove All

Implement a function `remove_all` that takes a `Link`, and a `value`, and remove any linked list node containing that value. You can assume the list already has at least one node containing `value` and the first element is never removed. Notice that you are not returning anything, so you should mutate the list.

#### Strategy

The implementation is straightfoward: If we run across a `.first` that is the same as `value`, then we shift the linked list. However, what makes the problem tricky is what we can do and what we can't do.

At first, we might think of the following implementation:

1. Base case: if the linked list is empty, returns nothing

In [ ]:
if not link:
    return

2. If `link.first` is equal to the `value`, then shift the link

In [ ]:
if link.first == value:
    link = link.rest

However, there's a flaw with this implementation: 

#### `link` is not mutated at all! 
For example, see the following,

In [ ]:
a = Link(2, Link(1, Link(3, Link(4, Link(5, Link(6, Link(7)))))))

def demo(link):
    if link.first == 2:
        link = link.rest
    return link

demo(a)

As we can see, the resultof calling `demo` on `a` is the result of the change. However, is `a` actually mutated?

In [ ]:
a

We can't mutate a linked list by assigning `link = link.rest`. To mutate a linked list, we need to shift the `link.rest` or its derivatives (`link.rest.rest`, `link.rest.rest.rest`, etc.) instead.

In [ ]:
a = Link(2, Link(1, Link(3, Link(4, Link(5, Link(6, Link(7)))))))

def demo(link):
    if link.first == 2:
        link.rest = link.rest.rest
    return link

demo(a)

In [ ]:
a

As we can see above, the linked list `a` is mutated!

Using the same idea but slightly modified:

**1.** If `link.rest` is empty, then returns nothing

In [ ]:
if not link.rest:
    return

**2.** If `link.second` is equal to value, then shift `link.rest`. Then call recursive `remove_all` on `link`.

In [ ]:
elif link.second == value:
    link.rest = link.rest.rest
    remove_all(link, value)

**3.** Otherwise, skip the linked list that we're currently looking at and move on to the next linked list, calling `remove_all` on it.

In [ ]:
remove_all(link.rest, value)

The implementation would look like the following:

In [ ]:
def remove_all(link, value):
    if not link.rest:
        return
    elif link.second == value:
        link.rest = link.rest.rest
        remove_all(link, value)
    else:
        remove_all(link.rest, value)

In [ ]:

"""Remove all the nodes containing value. Assume there exists some
    nodes to be removed and the first element is never removed.

    >>> l1 = Link(0, Link(2, Link(2, Link(3, Link(1, Link(2, Link(3)))))))
    >>> print(l1)
    <0 2 2 3 1 2 3>
    >>> remove_all(l1, 2)
    >>> print(l1)
    <0 3 1 3>
    >>> remove_all(l1, 3)
    >>> print(l1)
    <0 1>
    """
import doctest
doctest.testmod()

A shorter implementation as the following:

In [ ]:
def remove_all(link, value):
    if link.rest:
        remove_all(link.rest, value)
        if link.second == value:
            link.rest = link.rest.rest

## Q10: Mutable Mapping

Implement `deep_map_mut(fn, link)`, which applies a function `fn` onto all elements in the given linked list `link`. If an element is itself a linked list, apply `fn` to each of its elements, and so on.

The implementation should mutate the original linked list. Do not create any new linked list.

**Hint**: The built-in isinstance function may be useful.

In [ ]:
s = Link(1, Link(2, Link(3, Link(4))))
isinstance(s, Link)

In [ ]:
isinstance(s, int)

#### Strategy

The implementation is straightforward.

**1.** Base case: if the linked list is empty, then return nothing

In [ ]:
if not link:
    return

**2.** If we come across a nested linked list (the value of `link.first` is another linked list), then recursive call `deep_map_mut` on that `link.first`.

In [ ]:
if isinstance(link.first, Link):
    deep_map_mut(fn, link.first)

**3.** Otherwise, apply `fn` to the value in `link.first`, then at the end of the function definition, recursive call `deep_map_mut` on `link.rest`

In [ ]:
else:
    link.first = fn(link.first)
deep_map_mut(fn, link.rest)

The implementation would look like the following:

In [ ]:
def deep_map_mut(fn, link):
    if not link:
        return
    elif isinstance(link.first, Link):
        deep_map_mut(fn, link.first)
    else:
        link.first = fn(link.first)    
    deep_map_mut(fn, link.rest)

In [ ]:


"""Mutates a deep link by replacing each item found with the
    result of calling fn on the item.  Does NOT create new Links (so
    no use of Link's constructor)

    Does not return the modified Link object.

    >>> link1 = Link(3, Link(Link(4), Link(5, Link(6))))
    >>> deep_map_mut(lambda x: x * x, link1)
    >>> print(link1)
    <9 <16> 25 36>
    """
import doctest
doctest.testmod()

## Q11: Cycles

The `Link` class can represent lists with cycles. That is, a list may contain itself as a sublist.

In [ ]:
>>> s = Link(1, Link(2, Link(3)))
>>> s.rest.rest.rest = s
>>> s.rest.rest.rest.rest.rest.first
3

Implement `has_cycle`, that returns whether a linked list contains a cycle.

**Hint**: Iterate through the linked list and try keeping track of which `Link` objects you've already seen.

#### Strategy

One way to implement `has_cycle` is to use a list `[]` that keeps track of the `Link` object that Python has gone through so far.

In [ ]:
links = []

The implementation is as the following,

In [ ]:
links = []
while link is not Link.empty:
    if link in links:
        return True
    links.append(link)
    link = link.rest
return False

The idea is:

**1.** We loop through the `link` and we stop it when we reach an empty link. This makes sense because:
* If the `link` was a cycle, we'll eventually find a repeated pattern and it'll return `True*
* If the `link` was not a cycle, we'll eventually come across an empty link. Python will exit the `while` loop and will return `False`.

In [ ]:
while link is not Link.empty:

**2.** Check if `link` is within the list `links`. If yes, then the `list` is a cycle. Return `True`.

In [ ]:
if link in links:
    return True

**3.** As long as we haven't found a matching pattern, we add the current `link` object to the list `links`, then we shift `link` and repeat the loop.

In [ ]:
links.append(link)
link = link.rest

## Cycles - Challenge

As an extra challenge, implement `has_cycle_constant` with only constant space. 

#### Strategy

The base case is that if `link` is empty in the first place, it's definitely not a cycle. Then just return `False`

In [ ]:
if link is Link.empty:
    return False

Otherwise, we have a proper linked list.

Make 2 pointers where one of them shifts slower than the other.

In [ ]:
slow, fast = link, link.rest

With a proper linked list, we can use a `while` loop to check if a link is a cycle or not. We can use `fast` as the indicator (rather than using `slow`) since if the link is not a cycle, `fast` will become empty faster and Python will exit from the `while` loop quicker.

In [ ]:
while fast is not Link.empty:

The `while` loop keeps running as long as `fast` is not empty. Then as a way of checking if the link is not a cycle, we can check if `fast.rest` is empty.

In [ ]:
if fast.rest is Link.empty:
    return False

Or, we can check if `slow` is the same as `fast` or `fast.rest`. We can't check if `slow` is the same as `fast.rest.rest` or further than that since if it's not a cycle, `fast.rest.rest` might be an overkill from an empty link (e.g. the `rest` of an empty link)

In [ ]:
if slow is fast or slow is fast.rest 
    return True

Otherwise, we update `slow` to be `slow.rest` and `fast` to be `fast.rest.rest` Since `fast` supposed to shift faster!

In [ ]:
slow, fast = slow.rest, fast.rest.rest

The implementation would look like the following

In [ ]:
def has_cycle_constant(link):
    if link is Link.empty:
        return False
    slow, fast = link, link.rest
    while fast is not Link.empty:
        if fast.rest is Link.empty:
            return False
        elif slow == fast:
            return True
        slow, fast = slow.rest, fast.rest.rest

# Tree Practice

## Q12 - Reverse Other -- CONSULTED SOLUTION MANUAL

Write a function `reverse_other` that mutates the tree such that **labels** on every other (odd-depth) level are reversed.

In [ ]:
"""Mutates the tree such that nodes on every other (odd-depth) level
    have the labels of their branches all reversed.

    >>> t = Tree(1, [Tree(2), Tree(3), Tree(4)])
    >>> reverse_other(t)
    >>> t
    Tree(1, [Tree(4), Tree(3), Tree(2)])
    >>> t = Tree(1, [Tree(2, [Tree(3, [Tree(4), Tree(5)]), Tree(6, [Tree(7)])]), Tree(8)])
    >>> reverse_other(t)
    >>> t
    Tree(1, [Tree(8, [Tree(3, [Tree(5), Tree(4)]), Tree(6, [Tree(7)])]), Tree(2)])
    """
import doctest
doctest.testmod()

In [ ]:
def reverse_other(t):
    def reverse_helper(t, need_reverse):
        if t.is_leaf():
            return
        new_labs = [child.label for child in t.branches][::-1]
        for i in range(len(t.branches)):
            child = t.branches[i]
            reverse_helper(child, not need_reverse)
            if need_reverse:
                child.label = new_labs[i]
    reverse_helper(t, True)